#Text Classification with TensorFlow 


In [12]:
#Installing the dependencies of tensorflow
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [13]:
#Importing the tensorflow library and printing the version of tensorflow
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print("Version: ", tf.__version__)

Version:  2.5.0


In [25]:
#We are splitting in to training and testing datasets.60:40 ratio
#we are loading the dataset using Tensorflow
train_data, validation_data, test_data = tfds.load(name="imdb_reviews", split=('train[:60%]', 'train[60%:]','test'),as_supervised=True)
print("Training Data :",len(train_data))
print("Testing Data :",len(test_data))
print("Validation Data :",len(validation_data))

Training Data : 15000
Testing Data : 25000
Validation Data : 10000


In [27]:
#Printing first 10 samples from dataset
train_examples,train_labels = next(iter(train_data.batch(10)))
train_examples

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [28]:
#Displaying the train_label data
train_labels

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [36]:
#We are using the pre-trained model Tensorflow hub
embedd = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedd, input_shape=[],dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

Exception ignored in: <function CapturableResource.__del__ at 0x7f18692db200>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/tracking/tracking.py", line 277, in __del__
    self._destroy_resource()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 933, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 764, in _initialize
    *args, **kwds))
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py", line 3050, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/e

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [41]:
#Building a Sequential model on the complete dataset
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [44]:
#Compiling the model with Adam optimizer .
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [45]:
#Training the model using model.fit
model_text = model.fit(train_data.shuffle(10000).batch(512),epochs=20,validation_data=validation_data.batch(512),verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 98ms/step - loss: 0.7997 - accuracy: 0.5421 - val_loss: 0.6259 - val_accuracy: 0.6124
Epoch 2/20
30/30 [==============================] - 3s 90ms/step - loss: 0.5299 - accuracy: 0.6597 - val_loss: 0.5138 - val_accuracy: 0.7204
Epoch 3/20
30/30 [==============================] - 3s 95ms/step - loss: 0.4290 - accuracy: 0.7889 - val_loss: 0.4413 - val_accuracy: 0.7774
Epoch 4/20
30/30 [==============================] - 4s 105ms/step - loss: 0.3559 - accuracy: 0.8493 - val_loss: 0.3934 - val_accuracy: 0.8227
Epoch 5/20
30/30 [==============================] - 4s 101ms/step - loss: 0.3049 - accuracy: 0.8839 - val_loss: 0.3622 - val_accuracy: 0.8318
Epoch 6/20
30/30 [==============================] - 3s 95ms/step - loss: 0.2668 - accuracy: 0.9009 - val_loss: 0.3399 - val_accuracy: 0.8481
Epoch 7/20
30/30 [==============================] - 3s 96ms/step - loss: 0.2378 - accuracy: 0.9150 - val_loss: 0.3255 - val_accuracy: 0.8577
Epoch 8/20


In [48]:
#Printing the loss and accuracy of the model
results = model.evaluate(test_data.batch(512), verbose=2)
for names,val in zip(model.metrics_names, results):
  print("%s: %.3f" % (names, val))

49/49 - 3s - loss: 0.3439 - accuracy: 0.8589
loss: 0.344
accuracy: 0.859
